**Введение**

Подготовка данных является, на мой взгляд, сильно недооцененным этапом при разработке модели. Нет, важность этого момента подчеркивается повсеместно. Но я крайне редко встречал обоснование применения некоего метода к конкретным данным. "Импутируем средним", "применим PCA", "найдем outliers по стандартному отклонению". Я всё время задавался вопросом - а на каком основании? Этот вопрос поднимается крайне редно или не поднимается вобще, или вскользь. Best practicies, этого достаточно.


Мои поиски привели меня к Levels of mesurement. Приведу три иллюстрации.

![title](img/dt_03.png)

![title](img/dt_01.png)

![title](img/dt_02.png)

<a href=https://medium.com/@rndayala/data-levels-of-measurement-4af33d9ab51a>Источник.</a>

Как оказалось, тип данных, с которым я имею дело принципиально важен. Не любой анализ или преобразование можно применять к любым данным.<br>
Даже элементарное понимание дискретен предиктор или нет позволяет получить намного меньше шума при импутации KNN или MissingForest, просто округлив значения.<br>
Имея мизерный опыт в ML я уже столкнулся в известном датасете Pima Indians с нулевыми значениями предикторов, которые являлись NaN по сути - теперь понятно почему.<br>

Второй неутешительный момент - производительность.<br>
Посмотрев в замочную скважину на обрезанный, но реальный датасет, я понял что sklearn и остальные библиотеки начального (если можно так выразиться) уровня для реальной работы подходят не всегда.<br>

Предварительный анализ данных это однотипная и обязательная операция и естественной моей мыслью было неким образом автоматизировать этот процесс.<br>
И если вопрос производительности можно решать при помощи Dask и JAX (и библиотек на его основе) хотя-бы на уровне полной утилизации ресурсов одной локальной машины. То с данными ситуация оказалась несколько сложней.<br>
Оказалось, что вопрос определения с какими фичами в датасете я имею дело не тривиален. Датасеты всё больше, данных собирается больше, и за их конфеденциальностью следят всё сильнее. В том же Fraud Detection уже не спросишь что такое V277 и почему так много NaN. Насколько я понял это тенденция, а не частный случай.


На все выше написанное можно было-бы возразить - работает же, какие-то уровни, можно-нельзя. Все так делают и все работает, точность присутствует, места на kaggle имеют место быть и так далее.<br>
Меня эти сомнения посещали. Но потом я задумался - где работает? На сгенерированных датасетах или датасетах собранных сорок лет назад, которые теперь используются для обучения на любом ноутбуке?<br> 
Нет, меня такая формулировка не устраивает. 
1. Не могу судить, может "правильный" подход будет в реальной жизни привозить +0,001 к точности модели, может забирать 10% - я проверю. 
2. Я считаю, что обрабатывая данные не задумываясь о основаниях мы привносим в модель непонятный inductive bias. И можем получить еще более непонятный hidden inductive bias от модели. 
3. ИМХО, Science в слове DataScience как-бы обязывает хотя-бы попытаться понять что делаешь.

**Предварительная постановка задачи**

Стараясь трезво оценивать свои возможности, я не буду самонадеянно формулировать задачу "Буду делать AutoML".<br>
На первой итерации я хотел-бы иметь утилиту, которая выполняла следующие функции:
1. Избавляла от рутинных, однотипных операций по предварительному анализу.
2. Оптимизировала производительность. Желательно с возможностью подключения к GCP, Amazon кластерам "из коробки".
3. Проводила предварительный элементарный анализ фич. Дискретность, последовательность, отношение уникальных значений к общему их числу, разряженность.
4. Нужно попытаться хотя-бы на уровне интуиции сформулировать правила разделения данных. Хотя-бы определение категорий.